In [ ]:
# install keggle library
! pip install kaggle

In [ ]:
 # configuring kaggle.json file path
 ! mkdir -p ~/.kaggle
 ! cp kaggle.json ~/.kaggle/
 ! chmod 600 ~/.kaggle/kaggle.json


In [ ]:
# api to fetch dataset from kaggle
!kaggle datasets download -d kazanova/sentiment140



In [ ]:
# extracting the compressed dataset
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

importing the dependencies


In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
import matplotlib.pyplot as plt


In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
# printing the stopwords in English
print(stopwords.words('english'))

data processing


In [ ]:
# loading the data from csv file to pandas dataframe
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding = 'ISO-8859-1')

In [ ]:
# checking the number of rows and coloumns
twitter_data.shape

In [ ]:
# we here printing some rows of the dataframe
twitter_data.head(1600)

In [ ]:
# naming the coloums and reading the dataset again

column_names = ['target','ids','date','flag','user','text']
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names = column_names, encoding = 'ISO-8859-1')


In [ ]:
# checking the number of rows and coloumns
twitter_data.shape

In [ ]:
# printing first 5 rows of the dataframe
twitter_data.head()

In [ ]:
# counting the number of missing values in the dataset
twitter_data.isnull().sum()

In [ ]:
# checking the distribution of target coloumn
twitter_data['target'].value_counts()

convert the target or label i.e 0 1

In [ ]:
twitter_data.replace({'target':{4:1}}, inplace=True)

In [ ]:
# checking the distribution of target coloumn
twitter_data['target'].value_counts()

0 means negative tweets
1 means positive tweets

stemming- is the process of reducing the word to its root word ,
example: actor,actress,acting = act

In [ ]:
port_stem= PorterStemmer()

In [ ]:
def stemming(content):

  stemmed_content = re.sub('[^a-zA-Z]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [ ]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)  #takes long time 1.6 million tweets are there.


In [ ]:
twitter_data.head()

In [ ]:
print(twitter_data['stemmed_content'])

In [ ]:
print(twitter_data['target'])

In [ ]:
# separating the data and label
X= twitter_data['stemmed_content'].values
Y= twitter_data['target'].values

In [ ]:
print(X)

In [ ]:
print(Y)

splitting the data to training data and test data

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

In [ ]:
print(X.shape,X_train.shape,X_test.shape)

In [ ]:
print(X_train)

In [ ]:
print(X_test)

In [ ]:
# converting the textual data to numerical data

vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
print(X_train)

In [ ]:
print(X_test)

trainaing the machine learning model

Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model = LogisticRegression(max_iter=2000)

In [ ]:
model.fit(X_train,Y_train)

model evaluation

accuracy score

In [ ]:
# accuracy score on the trainning data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)

In [ ]:
print('accuracy score on the training data:',training_data_accuracy)

In [ ]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test, X_test_prediction)

In [ ]:
print('accuracy score on the test data:',test_data_accuracy)

model accuracy : 79.5 %

In [ ]:
cm = confusion_matrix(Y_test,X_test_prediction)

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Negative', 'Positive'])
disp.plot(cmap='Blues')
plt.title("Confusion Matrix")
plt.show()

In [ ]:
print(cm)

In [ ]:
# Predict on test data
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report

# Print Precision, Recall, and F1-Score
print(classification_report(Y_test, y_pred))

saving the trained model

In [ ]:
import pickle

In [ ]:
filename = 'tained_model.sav'
pickle.dump(model,open(filename,'wb'))

using the saved model for future predictions

In [ ]:
# loading the saved model
loaded_model = pickle.load(open('/content/tained_model.sav','rb'))

In [ ]:
X_new = X_test[200]
print(Y_test[200])
comment_text = twitter_data.iloc[X_test.indices[200]]['text']

prediction = loaded_model.predict(X_new)
print(prediction)
print(comment_text)

if(prediction[0]==0):
  print('Negative Tweet')
else:
  print('Positive Tweet')

In [ ]:
X_new = X_test[64]
print(Y_test[64])
comment_text = twitter_data.iloc[X_test.indices[64]]['text']
prediction = loaded_model.predict(X_new)
print(prediction)
print(comment_text)
if(prediction[0]==0):
  print('Negative Tweet')
else:
  print('Positive Tweet')

In [ ]:
X_new = X_test[15]
print(Y_test[15])
comment_text = twitter_data.iloc[X_test.indices[15]]['text']
prediction = loaded_model.predict(X_new)
print(prediction)
print(comment_text)
if(prediction[0]==0):
  print('Negative Tweet')
else:
  print('Positive Tweet')

In [ ]:
def preprocess(text):
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    text = text.split()
    text = [port_stem.stem(word) for word in text if word not in stopwords.words('english')]
    return ' '.join(text)

In [ ]:
# Take input from the user
user_tweet = input("enter new tweets: ")

# Preprocess the tweet
processed_tweet = preprocess(user_tweet)

# Converting  to TF-IDF
vectorized_tweet = vectorizer.transform([processed_tweet])


prediction = model.predict(vectorized_tweet)


if prediction[0] == 1:
    print("Sentiment: Positive Tweet 😊")
else:
    print("Sentiment: Negative Tweet 😠")
